## Solutions for Unit 5 Actions Part 2

<img src="../img/robotignite_logo_text.png"/>

## Index: 

* <a href="#SolutionExercise4-13">Solution Exercise 5.13</a>

## Solution Exercise 5.13 <p id="SolutionExercise4-13"></p>

<p style="background:#EE9023;color:white;">**Exercise 5.13**</p>

For this exercise, we will assume that our package is called **exercise_513**, our launch file is called **move_drone_square.launch**, and our Python file is called **move_drone_square.py**.

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Launch File: move_drone_square.launch** </p>

In [ ]:
<launch>
    <node pkg="exercise_513" type="move_drone_square.py" name="move_drone_square" output="screen" />
</launch>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Launch File: move_drone_square.launch** </p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Python File: move_drone_square.py** </p>

In [ ]:
#! /usr/bin/env python
import rospy
import time
import actionlib

from actionlib.msg import TestFeedback, TestResult, TestAction
from geometry_msgs.msg import Twist
from std_msgs.msg import Empty

class MoveSquareClass(object):
    
  # create messages that are used to publish feedback/result
  _feedback = TestFeedback()
  _result   = TestResult()

  def __init__(self):
    # creates the action server
    self._as = actionlib.SimpleActionServer("move_drone_square_as", TestAction, self.goal_callback, False)
    self._as.start()
    self.ctrl_c = False
    self.rate = rospy.Rate(10)
    
  
  def publish_once_in_cmd_vel(self, cmd):
    """
    This is because publishing in topics sometimes fails teh first time you publish.
    In continuos publishing systems there is no big deal but in systems that publish only
    once it IS very important.
    """
    while not self.ctrl_c:
        connections = self._pub_cmd_vel.get_num_connections()
        if connections > 0:
            self._pub_cmd_vel.publish(cmd)
            rospy.loginfo("Publish in cmd_vel...")
            break
        else:
            self.rate.sleep()
            
  # function that stops the drone from any movement
  def stop_drone(self):
    rospy.loginfo("Stopping...")
    self._move_msg.linear.x = 0.0
    self._move_msg.angular.z = 0.0
    self.publish_once_in_cmd_vel(self._move_msg)
        
  # function that makes the drone turn 90 degrees
  def turn_drone(self):
    rospy.loginfo("Turning...")
    self._move_msg.linear.x = 0.0
    self._move_msg.angular.z = 1.0
    self.publish_once_in_cmd_vel(self._move_msg)
    
  # function that makes the drone move forward
  def move_forward_drone(self):
    rospy.loginfo("Moving forward...")
    self._move_msg.linear.x = 1.0
    self._move_msg.angular.z = 0.0
    self.publish_once_in_cmd_vel(self._move_msg)
    
  def goal_callback(self, goal):
    # this callback is called when the action server is called.
    # this is the function that computes the Fibonacci sequence
    # and returns the sequence to the node that called the action server
    
    # helper variables
    r = rospy.Rate(1)
    success = True
    
    # define the different publishers and messages that will be used
    self._pub_cmd_vel = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
    self._move_msg = Twist()
    self._pub_takeoff = rospy.Publisher('/drone/takeoff', Empty, queue_size=1)
    self._takeoff_msg = Empty()
    self._pub_land = rospy.Publisher('/drone/land', Empty, queue_size=1)
    self._land_msg = Empty()
    
    # make the drone takeoff
    i=0
    while not i == 3:
        self._pub_takeoff.publish(self._takeoff_msg)
        rospy.loginfo('Taking off...')
        time.sleep(1)
        i += 1
    
    # define the seconds to move in each side of the square (which is taken from the goal) and the seconds to turn
    sideSeconds = goal.goal
    turnSeconds = 1.8
    
    i = 0
    for i in xrange(0, 4):
    
      # check that preempt (cancelation) has not been requested by the action client
      if self._as.is_preempt_requested():
        rospy.loginfo('The goal has been cancelled/preempted')
        # the following line, sets the client in preempted state (goal cancelled)
        self._as.set_preempted()
        success = False
        # we end the calculation of the Fibonacci sequence
        break
    
      # Logic that makes the robot move forward and turn
      self.move_forward_drone()
      time.sleep(sideSeconds)
      self.turn_drone()
      time.sleep(turnSeconds)
      
      # build and publish the feedback message
      self._feedback.feedback = i
      self._as.publish_feedback(self._feedback)
      # the sequence is computed at 1 Hz frequency
      r.sleep()
    
    # at this point, either the goal has been achieved (success==true)
    # or the client preempted the goal (success==false)
    # If success, then we publish the final result
    # If not success, we do not publish anything in the result
    if success:
      self._result.result = (sideSeconds*4) + (turnSeconds*4)
      rospy.loginfo('The total seconds it took the drone to perform the square was %i' % self._result.result )
      self._as.set_succeeded(self._result)
        
      # make the drone stop and land
      self.stop_drone()
      i=0
      while not i == 3:
          self._pub_land.publish(self._land_msg)
          rospy.loginfo('Landing...')
          time.sleep(1)
          i += 1
      
if __name__ == '__main__':
  rospy.init_node('move_square')
  MoveSquareClass()
  rospy.spin()

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Python File: move_drone_square.py** </p>

As you can see, in the above code we are using the seconds to move in each side of the square in order to define if the square will be bigger or smaller. So, for more seconds defined in the goal, the square will be bigger. This could be done in many ways, like checking the odometry to identify the distance the drone is actually moving, but that's quite more complex, and it's not the purpose of the exercise. The purpose of this Exercise is that you learn how to build a proper Action Server, and interact with the goal, feedback and result of the Action.

In order to test this code, you have to first launch the Action Server (which is defined in the aboce Python file). You can do that by using the following command:

In [ ]:
roslaunch exercise_513 move_drone_saquare.launch

Now, if you execute the following command:

In [ ]:
rostopic list | grep move_drone_square

You will see something like this:

<img src="../img/move_drone_square_as.png" width="600" />

This means you Action Server is up and ready to receive a goal.

So now, let's publish a goal to this Action Server. You can do this by two methods (as you saw in the Part 1 of the Actions notebooks):

* Creating an Action Client
* Publishing a goal directly through the action topics

For this case, let's use the 2nd method. You should then publish a message to the **move_drone_square_as/goal** topic, just like this:

<img src="../img/goal_square_sol.png" width="800" />

You will see the drone doing something like this:

<img src="../img/drone_square_sol.gif" width="600" />

Finally, you can check the **feedback** and **result** topics of the Action, to check if they're publishing the desired values.

In [ ]:
rostopic echo /move_drone_square_as/feedback

<img src="../img/feedback_square_sol.png" width="600" />

In [ ]:
rostopic echo /move_drone_square_as/result

<img src="../img/result_square_sol.png" width="600" />